In [114]:
import requests
from bs4 import BeautifulSoup

# Fetch the page content
url = "https://www.animeboston.com/schedule/index/2024"
soup = BeautifulSoup(requests.get(url).text, 'html.parser')

# Find all table rows containing the event details
events = []

for td in soup.select("td.schedule-event"):
    category_number = td.get("class")[1].split('-')[-1]  # Extract category number
    title = td.get("title")  # Get the title from the 'title' attribute
    description = td.get_text(strip=True)  # Extract the text description, remove extra spaces
    
    events.append({
        "category_number": category_number,
        "title": title,
        "title_table": description
    })

# Create a DataFrame from the list of events
events_df_soup = pd.DataFrame(events).drop_duplicates().sort_values(by='title', ascending=True)
events_df_soup['title'] = events_df_soup['title'].replace("", np.nan)  # Replace empty strings with NaN
events_df_soup['title'] = events_df_soup['title'].fillna(events_df_soup['title_table'])  # Fill NaN titles with descriptions
events_df_soup['category_number'] = events_df_soup['category_number'].astype(int)


display(events_df_soup)

,category_number,title,title_table
108,5,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","""Your Mom vs. the Noobs"" aka Otaku Mad Libs"
398,5,How to Frankenstein a Pattern for Cosplay,How to Frankens tein a Pattern for Cosplay
386,33,(18+) AMV Freeplay,(18+) AMV Freeplay (18+)
304,8,5-Star Pull: Genshin Impact Q&A,5-Star Pull: Genshin Impact Q&A
151,32,50% Off,50% Off
...,...,...,...
111,10,"Yoko Taro-verse: Sex, Death, and Violence","Yoko Taro-ver se: Sex, Death, and Violence ..."
118,1,Yu Yu Hakusho Marathon,Yu Yu Hakusho Marathon
97,30,hololive Meet: Karaoke Party,hololive Meet: Karaoke Party
240,9,hololive Meet: Saturday Panel,hololive Meet: Saturday Panel


In [117]:
# Extract legend items and build color map with data-value
legend_items = soup.select("div.schedule-legend label.schedule-category-label")

category_colors = {
    label.text.strip(): {
        'Category': label.text.strip(),
        'Color': label.get("style").split("background-color:")[1].strip(),
        'Data Value': label.find('input').get('data-value')
    }
    for label in legend_items if "background-color:" in label.get("style", "")
}

# Convert dictionary to DataFrame
category_colors_df = pd.DataFrame.from_dict(category_colors, orient='index')
category_colors_df['Data Value'] = category_colors_df['Data Value'].astype(int)
# fix missing accent in category name
category_colors_df['Category'] = category_colors_df['Category'].replace("Maid Cafe", "Maid Café")

category_colors_df.to_csv('Event_Categories.csv', index=False, encoding='utf-8-sig')

# Display the DataFrame
display(category_colors_df)


,Category,Color,Data Value
18+ Cosplay,18+ Cosplay,#AA99FF,35
18+ Event,18+ Event,#CC9966,12
18+ Fan Creations,18+ Fan Creations,#669966,33
18+ Fan Panel,18+ Fan Panel,#669900,10
18+ Featured Panel,18+ Featured Panel,#66CC00,15
18+ Gameshow,18+ Gameshow,#9900FF,11
18+ Guest Panel,18+ Guest Panel,#669999,17
18+ Video,18+ Video,#F7921B,3
21+ Event,21+ Event,#FF0000,27
Ball,Ball,#CC9900,40


In [118]:
all_events_df = pd.merge(events_df_soup, category_colors_df, left_on='category_number', right_on='Data Value', how= 'inner').sort_values(by=['category_number','title']).drop(columns=['category_number', 'Data Value'])
all_events_df

,title,title_table,Category,Color
7,A Certain Magical Index,A Certain Magical Index,Video,#66A3D2
8,A Certain Scientific Railgun,A Certain Scientif ic Railgun,Video,#66A3D2
22,Accel World,Accel World,Video,#66A3D2
25,Akudama Drive,Akudama Drive (PG-13),Video,#66A3D2
43,Appleseed 1988 OVA,Applesee d 1988 OVA (TVMA),Video,#66A3D2
...,...,...,...,...
158,IchiRockU,IchiRock U,Jam Zone,#CC0000
217,OFF-KNOWNEOUS,OFF-KNOW NEOUS,Jam Zone,#CC0000
223,Open Jam,Open Jam,Jam Zone,#CC0000
228,Paint by Numbers,Paint by Numbers,Jam Zone,#CC0000


In [119]:
def preprocess_tables(tables, ii, FirstColumnName='TimeSlot'):
    tables[ii].dropna(subset=[tables[ii].columns[0]], inplace=True)
    tables[ii] = tables[ii].iloc[:, :-1].copy()
    tables[ii].columns = [FirstColumnName] + tables[ii].columns[1:].tolist()
    #tables[ii].map(lambda x: x.replace('\n', '').strip() if isinstance(x, str) else x)

    # 24-hr military time
    vconvert = np.vectorize(lambda x: datetime.strptime(x, '%I:%M %p').strftime('%H:%M'))
    tables[ii][FirstColumnName] = vconvert(tables[ii][FirstColumnName])
    return tables[ii]

tables = pd.read_html(url)  # This reads the first two rows as column headers

# display cleaned version
#display(tables)

df = preprocess_tables(tables,0)
df

,TimeSlot,"(Hynes, Auditorium Events)","(Hynes, Ballroom A)","(Hynes, Ballroom B)","(Hynes, Fan Creations 312)","(Hynes, Maid Cafe)","(Hynes, Panel 202)","(Hynes, Panel 207)","(Hynes, Panel 208)","(Hynes, Panel 302)",...,"(Hynes, Video 306)","(Kings, Kings)","(Sheraton, Grand Ballroom)","(Sheraton, Panel Constitution)","(Sheraton, Panel Gardner)","(Sheraton, Panel The Fens)","(Sheraton, Republic Ballroom)","(Sheraton, RPG Riverway)","(Sheraton, Video Hampton)","(Sheraton, Workshop Fairfax)"
0,08:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN
1,08:15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN
2,08:30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN
3,08:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Soul Eater,NaN
4,09:00,NaN,NaN,NaN,AMV Genkis,NaN,NaN,NaN,NaN,NaN,...,Dual! Parallel Trouble Adventur e,NaN,NaN,NaN,NaN,NaN,NaN,Tabletop Session 1 - A Strange New World,Soul Eater,Let's $ew! Ditto!
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,00:45,NaN,Slumber Party,NaN,Fanservi ce & Ecchi AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
68,01:00,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
69,01:15,NaN,Slumber Party,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,01:30,NaN,NaN,NaN,Horror AMVs (18+),NaN,NaN,NaN,NaN,Taskmast er (18+),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
# add original row order as a helper column
df["RowOrder"] = df.index

# melt while preserving row order
long_df = df.melt(id_vars=["TimeSlot", "RowOrder"], 
                  var_name="Room", 
                  value_name="Event")

# drop rows without an event
events_df = long_df.dropna(subset=["Event"]).sort_values("RowOrder").drop(columns="RowOrder").reset_index(drop=True)

# cleaning up names
events_df.columns = events_df.columns.str.strip()
all_events_df.columns = all_events_df.columns.str.strip()

events_df = pd.merge(events_df, all_events_df, left_on='Event', right_on='title_table', how='inner').drop(columns=['Event']).rename(columns={'title':'Event','title_table':'Event_table'})
events_df

,TimeSlot,Room,Event,Event_table,Category,Color
0,08:00,"(Hynes, Video 306)",Dual! Parallel Trouble Adventure,Dual! Parallel Trouble Adventur e,Video,#66A3D2
1,08:00,"(Hynes, Video 210)",Lupin the 3rd: The Castle of Cagliostro,Lupin the 3rd: The Castle of Cagliost ro,Video,#66A3D2
2,08:00,"(Sheraton, Video Hampton)",Soul Eater,Soul Eater,Video,#66A3D2
3,08:15,"(Hynes, Video 210)",Lupin the 3rd: The Castle of Cagliostro,Lupin the 3rd: The Castle of Cagliost ro,Video,#66A3D2
4,08:15,"(Sheraton, Video Hampton)",Soul Eater,Soul Eater,Video,#66A3D2
...,...,...,...,...,...,...
742,00:15,"(Hynes, Ballroom A)",Slumber Party,Slumber Party,Event,#E8D37E
743,00:30,"(Hynes, Ballroom A)",Slumber Party,Slumber Party,Event,#E8D37E
744,00:45,"(Hynes, Ballroom A)",Slumber Party,Slumber Party,Event,#E8D37E
745,01:00,"(Hynes, Ballroom A)",Slumber Party,Slumber Party,Event,#E8D37E


In [121]:
# group by Event and aggregate TimeSlot into a set
df_grouped = events_df.groupby(['Event','Room','Category','Color'])['TimeSlot'].agg(list).reset_index()
df_grouped

,Event,Room,Category,Color,TimeSlot
0,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","(Sheraton, Panel Gardner)",Fan Panel,#00FF00,"[17:30, 17:45, 18:00, 18:15]"
1,50% Off,"(Hynes, Fan Creations 312)",Fan Creations,#CCFF66,"[20:45, 21:00, 21:15, 21:30, 21:45, 22:00, 22:..."
2,A Brief History of Anime at the Movies,"(Hynes, Panel 207)",Guest Panel,#00FFFF,"[16:30, 16:45, 17:00, 17:15]"
3,A Certain Magical Index,"(Hynes, Video 210)",Video,#66A3D2,"[10:00, 10:15, 10:30, 10:45, 11:00, 11:15, 11:..."
4,A Plus Size Cosplayer’s Survival Guide to Cost...,"(Sheraton, Workshop Fairfax)",Fan Panel,#00FF00,"[17:30, 17:45, 18:00, 18:15]"
...,...,...,...,...,...
126,Why People Love Japanese Snacks,"(Hynes, Panel 202)",Fan Panel,#00FF00,"[09:30, 09:45, 10:00, 10:15]"
127,Working in the JAV industry to anime voice act...,"(Hynes, Panel 208)",Guest Panel,#00FFFF,"[19:00, 19:15, 19:30, 19:45]"
128,"Worldweaving: Character Design, Clothing Histo...","(Hynes, Panel 309)",Fan Panel,#00FF00,"[11:00, 11:15, 11:30, 11:45]"
129,Yu Yu Hakusho Marathon,"(Sheraton, Video Hampton)",Video,#66A3D2,"[18:00, 18:15, 18:30, 18:45, 19:00, 19:15, 19:..."


In [122]:
df_grouped[df_grouped['Event']=='Maid Café']

,Event,Room,Category,Color,TimeSlot
71,Maid Café,"(Hynes, Maid Cafe)",Event,#E8D37E,"[12:00, 12:15, 12:30, 12:45, 13:00, 13:15, 13:..."


In [123]:
import pandas as pd

def split_event_to_subevents(df, event_col, time_col, room_col, chunk_size=3, target_event=None):
    """
    Splits a specific event into subevents by grouping its time slots into chunks.
    
    Parameters:
    - df: The DataFrame containing the events.
    - event_col: The name of the column containing event names.
    - time_col: The name of the column containing time slots.
    - room_col: The name of the column containing room information.
    - chunk_size: The number of time slots per subevent (default 3).
    - target_event: The specific event to split (e.g., 'Maid Café'). If None, all events are processed.
    
    Returns:
    - A DataFrame with subevents, grouped by event and room.
    """
    # If target_event is provided, filter only that event
    if target_event:
        df = df[df[event_col] == target_event]

    # Explode the TimeSlot column and sort by Room and TimeSlot
    exploded_df = (df.explode(time_col)
                   .sort_values(by=[room_col, time_col])
                   .reset_index(drop=True))

    # Create subevent labels based on chunking logic
    exploded_df['Subevent'] = exploded_df.groupby(room_col).cumcount() // chunk_size
    exploded_df['Subevent'] = exploded_df['Subevent'].apply(lambda x: f"{chr(65 + x)}")

    # Group by subevent and room, aggregate time slots into lists
    return (exploded_df
            .groupby([event_col, 'Subevent', 'Category', 'Color', room_col])[time_col]
            .agg(list)
            .reset_index())

# Example usage
# Assuming 'Event', 'TimeSlot', 'Room' are the column names in your DataFrame
subevent_df = split_event_to_subevents(df_grouped, 'Event', 'TimeSlot', 'Room', chunk_size=3, target_event="Maid Café")
subevent_df

,Event,Subevent,Category,Color,Room,TimeSlot
0,Maid Café,A,Event,#E8D37E,"(Hynes, Maid Cafe)","[12:00, 12:15, 12:30]"
1,Maid Café,B,Event,#E8D37E,"(Hynes, Maid Cafe)","[12:45, 13:00, 13:15]"
2,Maid Café,C,Event,#E8D37E,"(Hynes, Maid Cafe)","[13:30, 13:45, 14:00]"
3,Maid Café,D,Event,#E8D37E,"(Hynes, Maid Cafe)","[14:15, 14:30, 14:45]"
4,Maid Café,E,Event,#E8D37E,"(Hynes, Maid Cafe)","[15:00, 15:15, 15:30]"
5,Maid Café,F,Event,#E8D37E,"(Hynes, Maid Cafe)","[15:45, 16:00, 16:15]"
6,Maid Café,G,Event,#E8D37E,"(Hynes, Maid Cafe)","[16:30, 16:45, 17:00]"
7,Maid Café,H,Event,#E8D37E,"(Hynes, Maid Cafe)","[17:15, 17:30, 17:45]"


In [81]:
len(df_grouped)

131

### Notes

- Maid Cafe subevents are optional, but crucial to capture when repeating events
- So what is needed next is to have the event and timeslot taken out and scored with utility, combined with metadata such as the category number or whatnot.

In [82]:
# Drop rows for "Maid Café"
df_cleaned = df_grouped[df_grouped["Event"] != "Maid Café"]

# Add 'Event' column explicitly (already present)
# Combine
df_combined = pd.concat([df_cleaned, subevent_df], 
                        ignore_index=True)
df_combined

,Event,Room,TimeSlot,Subevent
0,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","(Sheraton, Panel Gardner)","[17:30, 17:45, 18:00, 18:15]",NaN
1,50% Off,"(Hynes, Fan Creations 312)","[20:45, 21:00, 21:15, 21:30, 21:45, 22:00, 22:...",NaN
2,A Brief History of Anime at the Movies,"(Hynes, Panel 207)","[16:30, 16:45, 17:00, 17:15]",NaN
3,A Certain Magical Index,"(Hynes, Video 210)","[10:00, 10:15, 10:30, 10:45, 11:00, 11:15, 11:...",NaN
4,A Plus Size Cosplayer’s Survival Guide to Cost...,"(Sheraton, Workshop Fairfax)","[17:30, 17:45, 18:00, 18:15]",NaN
...,...,...,...,...
133,Maid Café,"(Hynes, Maid Cafe)","[14:15, 14:30, 14:45]",D
134,Maid Café,"(Hynes, Maid Cafe)","[15:00, 15:15, 15:30]",E
135,Maid Café,"(Hynes, Maid Cafe)","[15:45, 16:00, 16:15]",F
136,Maid Café,"(Hynes, Maid Cafe)","[16:30, 16:45, 17:00]",G


In [83]:
# Unit Test, observe the duplicates! (They should have subevents)
df_combined[df_combined["Event"].duplicated(keep=False)]

,Event,Room,TimeSlot,Subevent
88,Room Clear,"(Hynes, Auditorium Events)","[11:30, 11:45, 15:00, 15:15]",NaN
89,Room Clear,"(Hynes, Ballroom A)","[14:30, 14:45, 16:00, 16:15, 21:00, 21:15]",NaN
90,Room Clear,"(Hynes, Ballroom B)","[19:00, 19:15]",NaN
91,Room Clear,"(Hynes, Fan Creations 312)","[15:15, 16:30, 20:30]",NaN
92,Room Clear,"(Hynes, Panel 208)","[11:00, 11:15, 12:30, 12:45, 14:30, 14:45]",NaN
93,Room Clear,"(Hynes, Panel 302)","[22:30, 22:45]",NaN
94,Room Clear,"(Hynes, Panel 309)","[15:00, 15:15]",NaN
95,Room Clear,"(Sheraton, Grand Ballroom)","[18:00, 18:15, 22:00, 22:15]",NaN
96,Room Clear,"(Sheraton, Panel Constitution)","[11:00, 11:15, 17:00, 17:15, 21:30, 21:45]",NaN
97,Room Clear,"(Sheraton, Republic Ballroom)","[16:30, 16:45, 19:00, 19:15, 20:30, 20:45]",NaN


In [84]:
exclude_from_scheduling = [
    "Room Clear",
    "Seating",
    "ID Check Seating (18+)"
]
df_final = df_combined[~df_combined["Event"].isin(exclude_from_scheduling)].reset_index(drop=True).copy()
df_final

,Event,Room,TimeSlot,Subevent
0,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","(Sheraton, Panel Gardner)","[17:30, 17:45, 18:00, 18:15]",NaN
1,50% Off,"(Hynes, Fan Creations 312)","[20:45, 21:00, 21:15, 21:30, 21:45, 22:00, 22:...",NaN
2,A Brief History of Anime at the Movies,"(Hynes, Panel 207)","[16:30, 16:45, 17:00, 17:15]",NaN
3,A Certain Magical Index,"(Hynes, Video 210)","[10:00, 10:15, 10:30, 10:45, 11:00, 11:15, 11:...",NaN
4,A Plus Size Cosplayer’s Survival Guide to Cost...,"(Sheraton, Workshop Fairfax)","[17:30, 17:45, 18:00, 18:15]",NaN
...,...,...,...,...
119,Maid Café,"(Hynes, Maid Cafe)","[14:15, 14:30, 14:45]",D
120,Maid Café,"(Hynes, Maid Cafe)","[15:00, 15:15, 15:30]",E
121,Maid Café,"(Hynes, Maid Cafe)","[15:45, 16:00, 16:15]",F
122,Maid Café,"(Hynes, Maid Cafe)","[16:30, 16:45, 17:00]",G


In [100]:
df_final

,Event,Room,TimeSlot,Subevent
0,"""Your Mom vs. the Noobs"" aka Otaku Mad Libs","(Sheraton, Panel Gardner)","[17:30, 17:45, 18:00, 18:15]",NaN
1,50% Off,"(Hynes, Fan Creations 312)","[20:45, 21:00, 21:15, 21:30, 21:45, 22:00, 22:...",NaN
2,A Brief History of Anime at the Movies,"(Hynes, Panel 207)","[16:30, 16:45, 17:00, 17:15]",NaN
3,A Certain Magical Index,"(Hynes, Video 210)","[10:00, 10:15, 10:30, 10:45, 11:00, 11:15, 11:...",NaN
4,A Plus Size Cosplayer’s Survival Guide to Cost...,"(Sheraton, Workshop Fairfax)","[17:30, 17:45, 18:00, 18:15]",NaN
...,...,...,...,...
119,Maid Café,"(Hynes, Maid Cafe)","[14:15, 14:30, 14:45]",D
120,Maid Café,"(Hynes, Maid Cafe)","[15:00, 15:15, 15:30]",E
121,Maid Café,"(Hynes, Maid Cafe)","[15:45, 16:00, 16:15]",F
122,Maid Café,"(Hynes, Maid Cafe)","[16:30, 16:45, 17:00]",G


### What's next

Tidying
- Need to have a way to link back and make sure that the events exploded list is consistent... could just explode this list

Scoring needed!
- Have a way to score this, probably write to csv, then score utility from 1 - 10, 10 being amazing, and 1 being absolutely not, this could be rated from multiple people and averaged among them (or take the minimum among all there)

Would be nice, best practice, can be later
- Afterward, there is a checking process for seeing unique constraints on the time slots, this could be a function or some such



In [85]:
df_final_exploded = df_final.explode('TimeSlot').sort_values(by=['Room', 'Event']).reset_index(drop=True)
df_final_exploded

,Event,Room,TimeSlot,Subevent
0,Cosplay Death Match,"(Hynes, Auditorium Events)",13:00,NaN
1,Cosplay Death Match,"(Hynes, Auditorium Events)",13:15,NaN
2,Cosplay Death Match,"(Hynes, Auditorium Events)",13:30,NaN
3,Cosplay Death Match,"(Hynes, Auditorium Events)",13:45,NaN
4,Cosplay Death Match,"(Hynes, Auditorium Events)",14:00,NaN
...,...,...,...,...
681,"Materia Girl: Women in Final Fantasy, On Scree...","(Sheraton, Workshop Fairfax)",22:45,NaN
682,Sketchbook Swap,"(Sheraton, Workshop Fairfax)",19:00,NaN
683,Sketchbook Swap,"(Sheraton, Workshop Fairfax)",19:15,NaN
684,Sketchbook Swap,"(Sheraton, Workshop Fairfax)",19:30,NaN


In [86]:
df_final_timeslotgrouped = df_final_exploded.groupby(['TimeSlot'])[['Event']].agg(set).reset_index()

In [87]:
#df_final_timeslotgrouped['Event'].iloc[0]

# {'A Certain Magical Index',
#  'AMV Genkis',
#  'Berklee Anime Band',
#  'Building a Manga Collecti on for a Library',
#  'Haibane Renmei',
#  'Irasshai mase! A Ramen Journey',
#  'Japanese Commerci als to Make You Laugh, Cry, and Think',
#  "Let's $ew! Ditto!",
#  'Opening Ceremoni es',
#  'SSSS. Gridman',
#  'Tabletop Session 1 - A Strange New World',
#  'The Turn of the Last Century: The Anime',
#  'Why People Love Japanese Snacks'}


In [88]:
# keep only first duplicates
# make Event lists hashable for deduplication
df_final_timeslotgrouped["Event_tuple"] = df_final_timeslotgrouped["Event"].apply(tuple)

# drop duplicates based on the Event list only, keeping the first time slot
df_dedupped_by_event = df_final_timeslotgrouped.drop_duplicates(subset="Event_tuple", keep="first")

# drop the helper column
df_dedupped_by_event = df_dedupped_by_event.drop(columns="Event_tuple")
df_dedupped_by_event


,TimeSlot,Event
0,00:00,{Slumber Party}
6,08:00,"{Soul Eater, Dual! Parallel Trouble Adventure ..."
10,09:00,"{AMV Genkis, Soul Eater, Dual! Parallel Troubl..."
12,09:30,"{AMV Genkis, Soul Eater, Dual! Parallel Troubl..."
14,10:00,"{The Turn of the Last Century: The Anime, AMV ..."
15,10:15,"{Original Fan Animation, The Turn of the Last ..."
16,10:30,"{Original Fan Animation, Pokémon Gathering, Th..."
18,11:00,"{Original Fan Animation, Pokémon Gathering, A ..."
20,11:30,"{Original Fan Animation, Fact or Fiction: Hist..."
21,11:45,{Fact or Fiction: Historical Events as Portray...


In [108]:
category_utility_base = {
    '18+ Cosplay': 7,
    '18+ Event': 6,
    '18+ Fan Creations': 6,
    '18+ Fan Panel': 5,
    '18+ Featured Panel': 6,
    '18+ Gameshow': 7,
    '18+ Guest Panel': 6,
    '18+ Video': 7,
    '21+ Event': 5,
    'Ball': 0, # assumed
    'Concerts': 7,
    'Cosplay Games': 6,
    'Event': 5,
    'Fan Creations': 6,
    'Fan Panel': 5,
    'Featured Artist': 5,
    'Featured Panel': 5,
    'Gameshow': 6,
    'Guest Panel': 5,
    'ID Check': 1,
    'Idol Events': 5,
    'Industry Panel': 5,
    'Jam Zone': 6,
    'Karaoke': 5,
    'Libraries & Education': 5,
    'Lolita & J-fashion': 5,
    'Maid Cafe': 7,
    'Premiere Video': 6,
    'Social Gatherings': 5,
    'Tabletop Gaming': 4,
    'Video': 4,
    'Workshop': 5,
}
category_utility_base

{'18+ Cosplay': 7,
 '18+ Event': 6,
 '18+ Fan Creations': 6,
 '18+ Fan Panel': 5,
 '18+ Featured Panel': 6,
 '18+ Gameshow': 7,
 '18+ Guest Panel': 6,
 '18+ Video': 7,
 '21+ Event': 5,
 'Ball': 0,
 'Concerts': 7,
 'Cosplay Games': 6,
 'Event': 5,
 'Fan Creations': 6,
 'Fan Panel': 5,
 'Featured Artist': 5,
 'Featured Panel': 5,
 'Gameshow': 6,
 'Guest Panel': 5,
 'ID Check': 1,
 'Idol Events': 5,
 'Industry Panel': 5,
 'Jam Zone': 6,
 'Karaoke': 5,
 'Libraries & Education': 5,
 'Lolita & J-fashion': 5,
 'Maid Cafe': 7,
 'Premiere Video': 6,
 'Social Gatherings': 5,
 'Tabletop Gaming': 4,
 'Video': 4,
 'Workshop': 5}

In [128]:
%run WebScraping.py